# Data Journalism Lesson 12: Working with spreadsheets

The spreadsheet is dead. Long live the spreadsheet.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async


In [ ]:
import micropip
await micropip.install('pyjanitor') 
await micropip.install('openpyxl')

In [ ]:
# Setup code for the notebook
import pandas as pd
import janitor  # For clean_names()
from IPython.display import display, HTML
import os

# --- Helper functions for grading exercises ---

def check_load_libraries(result):
    # Check if pandas and janitor are imported
    if 'pd' in globals() and 'janitor' in globals():
        display(HTML('<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> Pandas and Janitor are imported.</div>'))
    else:
        missing = []
        if 'pd' not in globals(): missing.append('pandas')
        if 'janitor' not in globals(): missing.append('janitor')
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> Make sure you import the following libraries: {", ".join(missing)}.</div>'))

def check_load_data_exercise(df):
    # Check if df is a DataFrame and has specific columns from the initial read (before skip)
    if isinstance(df, pd.DataFrame) and '...2' in df.columns and '...3' in df.columns:
         display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Great work!</strong> You imported the spreadsheet. Notice the messy headers initially.</div>'))
    elif isinstance(df, pd.DataFrame):
         display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Almost!</strong> Check the file path and ensure you loaded the data into a DataFrame named emitters.</div>'))
    else:
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> Make sure youve loaded the data into a pandas DataFrame.</div>'))

def check_head_display(result):
    # Simple check, assumes user ran .head()
    display(HTML('<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Code Ran.</strong> Make sure you see the first few rows of the DataFrame.</div>'))

def check_load_data2_exercise(df, expected_header_part):
    # Check if df is a DataFrame and the header looks correct after skipping
    if isinstance(df, pd.DataFrame) and expected_header_part in df.columns:
         display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> You skipped the first 3 rows successfully. The headers look much better.</div>'))
    elif isinstance(df, pd.DataFrame):
         display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Almost!</strong> Did you specify `skiprows=3` correctly? Check the resulting column names.</div>'))
    else:
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> Make sure your result is a pandas DataFrame.</div>'))

def check_load_data3_exercise(df):
    # In pandas, guess_max isn't a direct param. We check skip again.
     if isinstance(df, pd.DataFrame) and 'Facility Name' in df.columns:
         display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Code Ran.</strong> You correctly used `skiprows=3`. Pandas handles type guessing automatically.</div>'))
     else:
         display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> Ensure you are still using `skiprows=3`.</div>'))

def check_load_data4_exercise(df):
    # Check if df is a DataFrame and names are cleaned
    if isinstance(df, pd.DataFrame) and 'facility_name' in df.columns and 'total_reported_direct_emissions' in df.columns:
         display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Excellent!</strong> You skipped rows and cleaned the column names using `janitor.clean_names()`.</div>'))
    elif isinstance(df, pd.DataFrame) and 'Facility Name' in df.columns:
         display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Almost!</strong> Did you pipe the result of `read_excel` to `janitor.clean_names()`?</div>'))
    else:
        display(HTML('<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> Make sure you are reading the file with `skiprows=3` and then cleaning the names.</div>'))

# --- Load Data & Define Variables ---
# Define state (e.g., Minnesota 'MN' like previous notebook, or 'NE' like Rmd default)
state_abbr = "MN" # Or choose another state like "NE"
state_name = "Minnesota"

# Path to the states lookup file (assuming it's in _static)
excel_file_path = "../_static/greenhouse-gas-emitters/ghgp_data_2022.xlsx" # Path relative to notebook

# Load and clean data for use in text
try:
    clean_emitters_df = pd.read_excel(excel_file_path, skiprows=3).clean_names()
    state_emitters_count = clean_emitters_df[clean_emitters_df['state'] == state_abbr].shape[0]
except FileNotFoundError:
    print(f"Error: Emitters Excel file not found at {excel_file_path}")
    print("Please ensure the file exists and the path is correct.")
    clean_emitters_df = pd.DataFrame() # Create empty df to avoid errors later
    state_emitters_count = 0
except Exception as e:
    print(f"An error occurred while loading emitters data: {e}")
    clean_emitters_df = pd.DataFrame()
    state_emitters_count = 0

In [ ]:
from pyodide.http import pyfetch
import pandas as pd
from io import BytesIO

# Fetch the file asynchronously
response = await pyfetch(url="../_static/greenhouse-gas-emitters/ghgp_data_2022.xlsx", method="GET")
data_bytes = await response.bytes()
excel_data = BytesIO(data_bytes)


In [ ]:
from myst_nb import glue

# Glue variables for use in markdown cells
glue("state_name", state_name, display=False)
glue("state_abbr", state_abbr, display=False)
glue("state_emitters_count", f"{state_emitters_count:,}", display=False)

## The Goal

In this lesson, you'll learn how to pull spreadsheets into dataframes, a common challenge in data journalism. The spreadsheet is still by far the most common data analysis tool, in spite of all of its limitations. By the end of this tutorial, you'll understand two methods for converting spreadsheets into dataframes using the most common spreadsheet platforms: Excel and Google Sheets. You'll also practice cleaning and organizing the extracted data.

## What is Data Journalism?

Ask just about any data journalist what tool they started with and you'll almost universally get the same answer: The spreadsheet. For the old timers, spreadsheets were like magic back when computers were relatively new and the idea of using data to do journalism was a form of punk rock -- rebellious almost.

Coulter Jones, now a data reporter at Bloomberg News, started his data journalism career in college, where he learned spreadsheets in a class.

"When you're in college or you're around people who are doing this, it's easy to compare yourself to people more advanced and feeling like, I don't know what I'm doing," he said. "And then you would work with colleagues who you literally see them counting paper or going through and doing like a control F or something, you know, and it's like, what are you doing?

"Then you sort of realize that like almost every story is about the most or the least. I mean, it's that basic. It's like, who did the most? Where are the most fines going? Who raised the most money this campaign cycle?"

It sounds simple, but those kinds of basic questions drove a project Jones worked on that won a Pulitzer Prize in 2023 about federal officials trading stocks in companies they regulated. The first questions editors had? Who owns certain stocks? (A simple filter). Who has the most money? (A group by and summarize).

Spreadsheets are everywhere in business and government. They're relatively easy to use -- you can learn 80-90 percent of what you need to learn in a day -- and extremely flexible for something organized into rows and columns. As spreadsheets have evolved, developers have added ways to add color and type and even charts made with few clicks.

That has given a lot of spreadsheet users delusions of grandeur, who now want to create pretty spreadsheets instead of useful spreadsheets.

It's likely in your time as a data journalist you'll have to use spreadsheets. If you've ever been to a NICAR conference, you'll note there are dozens of spreadsheet classes. The reason this textbook doesn't involve spreadsheets, outside of this lesson, is because they're out of step with modern data science and modern journalism.

Modern data science/data journalism requires transparency. What does that mean? Notebooks, with code and data, are publicly available so anyone can look at what you did and decide for themselves if you did a good job or not.

Can you do that with a spreadsheet? My argument is no. There's no way to audit your work, there's no way to reliably show how you did what you did step by step. A good faith argument is that it can be done, it's just very hard. If you miss a step, your analysis will appear wrong to someone coming behind.

"There's a difference between doing a quick daily story where you're just doing a pivot table or something and you write it and it's done ... versus like a longer term project or investigation and you go through that fact check and someone says, well, why are we using this number and not this other one?" Jones told me.

So how can you apply modern data science/data journalism principles in a world still awash in spreadsheets? You can pull that spreadsheet into Python using pandas.

## The Basics

How do you pull a spreadsheet into Python? Well, depending on the sheet, it might be *very* easy. But if the data provider you are working with tried to make it pretty, it can get a little worse.

One near-universal truth you should keep in mind as you get better at programming: If you have a problem, someone else probably had it first and they very likely made a library that solves that problem. Do you think there are developers in the world who wanted to pull a spreadsheet into Python? Indeed there has been.

The library we're going to work with today is `pandas`, specifically its `read_excel` function. You don't have to be a genius to figure out what this function does. We're going to load all of our libraries first and then contend with the data. One other library we're going to need -- `pyjanitor` (imported as `janitor`). Think of `janitor` as the library that's going to have to clean up all this spreadsheet glitter someone thought would make it sparkle.

In [ ]:
import pandas as pd
import janitor

The dataset we're going to work with is from a U.S. Environmental Protection Agency program called the [Greenhouse Gas Reporting Program](https://www.epa.gov/ghgreporting/data-sets). The program collects greenhouse gas emissions from "large emitting facilities, suppliers of fossil fuels and industrial gases that result in GHG emissions when used, and facilities that inject carbon dioxide underground." These facilities, all over the country, are interesting data to look at when reporting on climate change at a local and state level and ... come in an Excel spreadsheet.

If you were to download the file and open it, you'd see an immediate problem: The first row of the spreadsheet is not the header row. In every dataset we've used so far, the first row is the header row, because that's what `read_csv` expects. It knows that most datasets have a header row, and it's first, so it takes that row and makes it the header. The second row in the csv is the first row of data. Here's what your spreadsheet looks like:

```{image} ../figures/spreadsheet1.png
:alt: Screenshot of Excel spreadsheet showing header info in first few rows
:width: 100%
:align: center
```

See how the first three rows are notes? What do we do about that?

First, we're going to use a function in `pandas` that should look and act *very* familiar: It's `pd.read_excel`. It does what you think it does.

In [ ]:
emitters = pd.read_excel(excel_data)
emitters.head()

That ... doesn't look familiar at all.

What does `emitters` look like?

In [ ]:
emitters.head()

It's a mess. First of all, our headers are all named `...2`, `...3`, `...4` and so on (pandas generates default names when the header row is problematic or missing). You can see what looks more like a header row in row 3 (index 2), which would actually be row 4 of the Excel sheet since `read_excel` tried to use row 1 (index 0) as a header row.

All along, we've been using `read_csv` with the defaults -- first row is the header, guess what each column is based on the first few hundred rows, no special processing instructions. This is what it looks like when those things aren't the case.

What we need to be able to tell `read_excel` is that it should just ignore those first three rows and use the 4th row (index 3 in 0-based indexing) as the header.

Handy enough: `read_excel` (and `read_csv`) both have a `skiprows` parameter. You can tell it to skip a certain number of rows *before* the header row. Our header row is in row 4 (Excel row number), so we need to skip the 3 rows above it.

### Exercise 1: Skipping

Use the `skiprows` parameter in `pd.read_excel` to skip the first 3 rows.

In [ ]:
emitters = pd.read_excel(excel_data, skiprows=____)

print(emitters.head())

check_load_data2_exercise(emitters, 'Facility Name')

Now we're getting something else we haven't seen -- import errors. *[Note: In pandas, type inference issues often manifest as columns with `object` dtype containing mixed types, or sometimes errors during later operations, rather than explicit warnings.]* What that *could* mean is based on the first N rows pandas samples, it's expecting one type of data, but it's getting a different kind later on. That might mean the first rows were numbers, but now there's text in the column and that's creating problems. It might mean the first rows had good dates and now they've gone bad. The point is, the guesses can sometimes be wrong.

Sometimes this is a major problem. Often, pandas handles mixed types by assigning the `object` dtype, which can be less efficient but generally works. If specific type issues arise, you might need to specify `dtype` options in `read_excel`.

For now, let's assume pandas' default guessing is sufficient and move on. We successfully skipped the rows.

### Guessing

If you encountered type errors (which we didn't explicitly see here, but could happen with other messy sheets), you might need to:
1.  Read the file without type inference (`dtype=object`) and convert types later.
2.  Specify the correct `dtype` for problematic columns directly in `read_excel`.
3.  Load the data and then use functions like `pd.to_numeric`, `pd.to_datetime` with `errors='coerce'` to fix types.

One last thing we can do that we haven't done before -- there is nothing stopping you from chaining on other commands to your read step. In fact, you could do everything we've done so far in one single block, going from `pd.read_excel(...)` -> `.query(...)` -> `.groupby(...)` -> `.agg(...)` and get output. It makes sense to separate your steps so you can diagnose problems where they are happening, but it's all very feasible to do everything in one giant code blob.

Sometimes, adding a little more to your read step will make your life a lot easier later. You probably haven't noticed because we haven't really looked, but some of the column names in this data are really gnarly from the standpoint of Python's typical variable naming conventions (and pandas best practices). For example, one column is called "Total reported direct emissions" which has spaces in it that are awkward to work with (requiring `df['Total reported direct emissions']` syntax). Another is "CO2 emissions (non-biogenic)", which adds parens to our column name.

Good thing you learned about `janitor.clean_names()` in Chapter 9. That would come in really handy here.

### Exercise 3: Cleaning

Chain the `janitor.clean_names()` function after reading the Excel file (with `skiprows`).

In [ ]:
emitters = (
    pd.read_excel(excel_data, skiprows=____)
    .____()
)

print(emitters.head())

check_load_data4_exercise(emitters)

Looks a lot like what we're used to, yes? You've now got a dataset you can run queries on, like asking how many of these are in {glue:text}`state_name`. There are {glue:text}`state_emitters_count`, if you're curious.

One thing to know: Spreadsheets *can* contain more than one sheet in them. This one actually contains 11 of them -- most of them subsets of the data, but some are completely different formats and structures. `pd.read_excel` by default just grabs sheet 1 (using a 0-based index, so `sheet_name=0`), unless you tell it otherwise. To do so, you'd just add `sheet_name=1` inside `read_excel` to get sheet 2, `sheet_name='SheetName'` to get a specific sheet by name, or `sheet_name=None` to get a dictionary of DataFrames, one for each sheet.

## Pulling data from Google Sheets

Part of the reason why Excel spreadsheets are everywhere is because it's the spreadsheet program that's stood the test of time. Microsoft first launched it in 1985 to compete with another product that no longer exists. Excel won.

But in the internet era, another spreadsheet platform is giving Excel a run for its money. Most current college students won't remember a time without Google Sheets, launched in 2006, and for most of them, because of Chromebooks in high school, Sheets was the first spreadsheet platform they'd ever seen.

That it comes in at the low, low price of free means a growing number of providers are using it to release data, particularly at the state and local levels.

Because of limitations in the tutorial data and how Google handles authorizations, I can't directly demonstrate it here, but there are several Python libraries like `gspread` and `pandas` itself (using specific URLs) that handle reading Google Sheets into a dataframe. The concepts are similar, but authentication is usually the trickiest part, often requiring API keys or OAuth2.

If you have a **publicly published** Google Sheet (specifically, published to the web as a CSV), you can often read it directly with `pd.read_csv` using a specially formatted URL.

For sheets requiring authentication, `gspread` is a popular choice:

```python
# Example using gspread (requires setup and authentication)
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd
import janitor

# --- Authentication Setup (replace with your actual credentials) ---
scope = ['https://www.googleapis.com/auth/spreadsheets.readonly']
creds_file = 'path/to/your/credentials.json' # You need to get this from Google Cloud
creds = Credentials.from_service_account_file(creds_file, scopes=scope)
client = gspread.authorize(creds)
# --- End Authentication Setup ---

# Open the Google Sheet by URL or title
sheet_url = 'YOUR_GOOGLE_SHEET_URL_HERE'
spreadsheet = client.open_by_url(sheet_url)

# Select a specific worksheet (e.g., the first one)
worksheet = spreadsheet.get_worksheet(0) # 0-based index

# Get all values as a list of lists and convert to DataFrame
data = worksheet.get_all_values()
headers = data.pop(0) # Assumes first row is header
gsheet_df = pd.DataFrame(data, columns=headers)

# Now you can apply skiprows-like logic (if needed) and clean names
# Example: If header was actually row 4 (index 3)
# data = worksheet.get_all_values()
# headers = data[3] # Get header from 4th row
# actual_data = data[4:] # Get data from 5th row onwards
# gsheet_df = pd.DataFrame(actual_data, columns=headers)
# gsheet_df = gsheet_df.clean_names()

print(gsheet_df.head())
```

The key takeaway is that while the tool changes, the core data manipulation concepts (handling headers, cleaning names) remain the same once the data is in a pandas DataFrame.

## The Recap

Throughout this lesson, you've mastered essential techniques for working with spreadsheet data in Python using pandas. You've learned to use `pd.read_excel()` to import Excel files, handling challenges like skipping rows using `skiprows`. You've also explored how to clean column names using the `pyjanitor` package's `clean_names()` function and discussed how to approach accessing data from Google Sheets using libraries like `gspread`. Remember, while spreadsheets are ubiquitous in data journalism, importing them into pandas allows for more transparent, reproducible analysis within your Python environment.

## Terms to Know

- **`pandas`**: A fundamental Python library for data manipulation and analysis.
- **`pd.read_excel()`**: A function in the `pandas` library used to read Excel files (.xls, .xlsx) into a DataFrame.
- **`skiprows`**: A parameter in `pd.read_excel()` (and other pandas read functions) that allows you to ignore a specified number of rows at the beginning of a spreadsheet before the header.
- **`sheet_name`**: A parameter in `pd.read_excel()` to specify which sheet(s) to read from an Excel file (by 0-based index, name, or `None` for all).
- **`pyjanitor`**: A Python library providing convenient functions for cleaning data, including `clean_names()`.
- **`janitor.clean_names()`**: A function from the `pyjanitor` package that standardizes column names (e.g., converts to snake_case) for easier data manipulation.
- **`gspread`**: A Python library that enables reading and writing Google Sheets directly from Python (often requires authentication setup).
- **Excel**: A spreadsheet program developed by Microsoft, widely used for data storage and analysis.
- **Google Sheets**: A web-based spreadsheet program offered by Google, part of the Google Workspace suite.